<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Solving Tasks Requiring Web Info

AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to perform tasks which require acquiring info from the web:
* discuss a paper based on its URL.
* discuss about the stock market.

Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "TERMINATE", the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and return the execution results (success or failure and corresponding outputs) to `AssistantAgent`, and prompt the user for feedback when the task is finished. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent`.

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install pyautogen and docker:
```bash
pip install pyautogen docker
```

In [1]:
# %pip install "pyautogen>=0.2.3" docker

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["mistal", "codellama","qwen", "llama2",  "gemma"],
    },
)

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4-32k',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt4',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
    {
        'model': 'gpt-4-32k-0314',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for human feedback when it receives a "TERMINATE" signal from the assistant agent.

In [3]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

## Example Task: Paper Talk from URL

We invoke the `initiate_chat()` method of the user proxy agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the assistant agent sends a "TERMINATE" signal at the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the assistant agent on behalf of the user.

In [4]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
Who should read this paper: https://arxiv.org/abs/2308.08155
""",
)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):


Who should read this paper: https://arxiv.org/abs/2308.08155


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


assistant (to user_proxy):

To read this paper, the following audience would be most interested:

1. **Researchers and Scientists**: The paper likely presents original research or findings in a field such as computer science, physics, or engineering.

2. **PhD Students and Postdocs**: These individuals are at the stage of conducting in-depth research and may find value in understanding the presented work.

3. **Academy Professors and Tenure Track Researchers**: These professionals are likely to be interested in reviewing new research in their field, as it could inform their teaching or future projects.

4. **Industry Professionals and Students**: Although not the primary target audience, industry professionals and students who are interested in learning about new developments in a specific technology may also find this paper relevant.




--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.


...

assistant (to user_proxy):

TERMINATE


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Please give feedback to assistant. Press enter or type 'exit' to stop the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


ChatResult(chat_id=None, chat_history=[{'content': '\nWho should read this paper: https://arxiv.org/abs/2308.08155\n', 'role': 'assistant'}, {'content': 'To read this paper, the following audience would be most interested:\n\n1. **Researchers and Scientists**: The paper likely presents original research or findings in a field such as computer science, physics, or engineering.\n\n2. **PhD Students and Postdocs**: These individuals are at the stage of conducting in-depth research and may find value in understanding the presented work.\n\n3. **Academy Professors and Tenure Track Researchers**: These professionals are likely to be interested in reviewing new research in their field, as it could inform their teaching or future projects.\n\n4. **Industry Professionals and Students**: Although not the primary target audience, industry professionals and students who are interested in learning about new developments in a specific technology may also find this paper relevant.\n\n\n', 'role': 'us

## Example Task: Chat about Stock Market

In [5]:
user_proxy.initiate_chat(
    assistant,
    message="""Show me the YTD gain of 10 largest technology companies as of today.""",
)

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):

Show me the YTD gain of 10 largest technology companies as of today.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


assistant (to user_proxy):

To get the YTD gain of the 10 largest technology companies, we need to access real-time data or a reliable API that provides this information.

Assuming we have access to such an API, here's a Python script using `pandas` to fetch and calculate the YTD gains:

```python
import pandas as pd

# Assuming you have an API endpoint for fetching company data
url = 'https://api.example.com/technology-companies'

# Fetch data from the API
data = pd.read_json(url)

# Sort by market capitalization (assumed to be the YTD gain) in descending order
largest_companies = data.sort_values('market_cap', ascending=False)[:10]]

# Calculate YTD gain for each company
ytd_gains = largest_companies['market_cap'].pct_change()

# Print the top 10 YTD gain companies along with their gains
print("Top 10 YTD Gain Technology Companies:")
for index, row in ytd_gains.head(10).iterrows():
    company_name = data.loc[index, 'company_name']]
    gain_percentage = round(row * 100), 2)
    
   

No default IOStream has been set, defaulting to IOConsole.


...

No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


assistant (to user_proxy):

Dear [User's Name],

I'm thrilled to see that you've successfully resolved the syntax issue in your Python code. That's great news and a testament to your problem-solving skills.

If you need any further assistance or have questions about data manipulation or analysis, please feel free to ask. I'm here to help and provide guidance whenever needed.

Remember, collaboration is key to overcoming challenges. If you encounter similar issues in the future, don't hesitate to reach out for support.

Congratulations on resolving the issue! I wish you all the best with your Python projects. If there's anything else that comes up, please let me know.

Warm regards,
[Your Name]


--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


Please give feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': 'Show me the YTD gain of 10 largest technology companies as of today.', 'role': 'assistant'}, {'content': 'To get the YTD gain of the 10 largest technology companies, we need to access real-time data or a reliable API that provides this information.\n\nAssuming we have access to such an API, here\'s a Python script using `pandas` to fetch and calculate the YTD gains:\n\n```python\nimport pandas as pd\n\n# Assuming you have an API endpoint for fetching company data\nurl = \'https://api.example.com/technology-companies\'\n\n# Fetch data from the API\ndata = pd.read_json(url)\n\n# Sort by market capitalization (assumed to be the YTD gain) in descending order\nlargest_companies = data.sort_values(\'market_cap\', ascending=False)[:10]]\n\n# Calculate YTD gain for each company\nytd_gains = largest_companies[\'market_cap\'].pct_change()\n\n# Print the top 10 YTD gain companies along with their gains\nprint("Top 10 YTD Gain Technology Companie